# Project 3 - Part 1

## Business problem

- Produce a MySQL database on movies from a subset of IMDB's publicly available dataset.
- Use this database to analyze what makes a movie successful.
- Provide recommendations to the stakeholder on how to make a successful movie.

## Requirements

- Download several files from IMDB’s movie data set.
- Exclude any movie with missing values for genre or runtime.
- Include only full-length movies (titleType = "movie").
- Include only fictional movies (not from documentary genre).
- Include only movies that were released 2000 - 2021 (include 2000 and 2021).
- Include only movies that were released in the United States.

## Imports

In [30]:
import pandas as pd
import numpy as np
import os

## Load the data

In [31]:
akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"

In [32]:
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [33]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [34]:
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

## Inspect and filter the data: Title AKAs

**Data dictionary**

- titleId (string) - a tconst, an alphanumeric unique identifier of the title
- ordering (integer) – a number to uniquely identify rows for a given titleId
- title (string) – the localized title
- region (string) - the region for this version of the title
- language (string) - the language of the title
- types (array) - Enumerated set of attributes for this alternative title. E.g. "alternative", "dvd", "festival", "tv", etc.
- attributes (array) - Additional terms to describe this alternative title, not enumerated
- isOriginalTitle (boolean) – 0: not original title; 1: original title

In [35]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37759203 entries, 0 to 37759202
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.3+ GB


In [36]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [37]:
print(f'There are {akas.shape[0]} rows and {akas.shape[1]} columns in the title_akas dataset.')

There are 37759203 rows and 8 columns in the title_akas dataset.


In [38]:
## Align datatypes
akas['isOriginalTitle'] = akas['isOriginalTitle'].astype(bool)

In [39]:
## Replace "\N" with np.nan
akas.replace({'\\N':np.nan}, inplace=True)

In [40]:
## Keep only US movies
akas = akas[akas['region'] == 'US']

In [41]:
## Cleaned and filtered data
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1477044 entries, 5 to 37758947
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1477044 non-null  object
 1   ordering         1477044 non-null  int64 
 2   title            1477044 non-null  object
 3   region           1477044 non-null  object
 4   language         4208 non-null     object
 5   types            985576 non-null   object
 6   attributes       47725 non-null    object
 7   isOriginalTitle  1477044 non-null  bool  
dtypes: bool(1), int64(1), object(6)
memory usage: 91.6+ MB


## Inspect and filter the data: Title Basics

**Data dictionary**

- tconst (string) - alphanumeric unique identifier of the title
- titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
- primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
- originalTitle (string) - original title, in the original language
- isAdult (boolean) - 0: non-adult title; 1: adult title
- startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year
- endYear (YYYY) – TV Series end year. ‘\N’ for all other title types
- runtimeMinutes – primary runtime of the title, in minutes
- genres (string array) – includes up to three genres associated with the title

In [42]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10311739 entries, 0 to 10311738
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 708.1+ MB


In [43]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [44]:
print(f'There are {basics.shape[0]} rows and {basics.shape[1]} columns in the title_basics dataset.')

There are 10311739 rows and 9 columns in the title_basics dataset.


In [45]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [46]:
## Replace "\N" with np.nan
basics.replace({'\\N':np.nan}, inplace=True)

In [47]:
## Eliminate movies that are null for runtimeMinutes
basics.dropna(subset=['runtimeMinutes'], inplace=True)

In [48]:
## Eliminate movies that are null for genre
basics.dropna(subset=['genres'], inplace=True)

In [49]:
## Keep only titleType==Movie
basics = basics[basics['titleType'] == 'movie']

In [50]:
## Keep startYear 2000-2022
basics = basics[(basics['startYear'] >= '2000') & (basics['startYear'] < '2022')]

In [51]:
## Eliminate movies that include "Documentary" in genre
is_documentary = basics['genres'].str.contains('documentary', case=False)
basics = basics[~is_documentary]

In [52]:
## Keep only US movies
us_movies = basics['tconst'].isin(akas['titleId'])
basics = basics[us_movies]

In [55]:
## Align datatypes
basics['isAdult'] = basics['isAdult'].astype(bool)
basics['runtimeMinutes'] = pd.to_numeric(basics['runtimeMinutes'], errors='coerce')

In [56]:
## Cleaned and filtered data
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82235 entries, 34800 to 10311505
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          82235 non-null  object
 1   titleType       82235 non-null  object
 2   primaryTitle    82235 non-null  object
 3   originalTitle   82235 non-null  object
 4   isAdult         82235 non-null  bool  
 5   startYear       82235 non-null  int32 
 6   endYear         0 non-null      object
 7   runtimeMinutes  82235 non-null  int64 
 8   genres          82235 non-null  object
dtypes: bool(1), int32(1), int64(1), object(6)
memory usage: 5.4+ MB


In [57]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34800,tt0035423,movie,Kate & Leopold,Kate & Leopold,True,2001,NaN,118,"Comedy,Fantasy,Romance"
61110,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,True,2020,NaN,70,Drama
67662,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,True,2018,NaN,122,Drama
80547,tt0082328,movie,Embodiment of Evil,Encarnação do Demônio,True,2008,NaN,94,Horror
86789,tt0088751,movie,The Naked Monster,The Naked Monster,True,2005,NaN,100,"Comedy,Horror,Sci-Fi"


## Inspect and filter the data: Title Ratings

**Data dictionary**

- tconst (string) - alphanumeric unique identifier of the title
- averageRating – weighted average of all the individual user ratings
- numVotes - number of votes the title has received

In [58]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1368573 entries, 0 to 1368572
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1368573 non-null  object 
 1   averageRating  1368573 non-null  float64
 2   numVotes       1368573 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 31.3+ MB


In [59]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2007
1,tt0000002,5.8,269
2,tt0000003,6.5,1911
3,tt0000004,5.5,178
4,tt0000005,6.2,2692


In [60]:
print(f'There are {ratings.shape[0]} rows and {ratings.shape[1]} columns in the title_basics dataset.')

There are 1368573 rows and 3 columns in the title_basics dataset.


In [61]:
## Replace "\N" with np.nan
ratings.replace({'\\N':np.nan}, inplace=True)

In [62]:
## Keep only US movies
us_movies = ratings['tconst'].isin(akas['titleId'])
ratings = ratings[us_movies]

In [63]:
## Cleaned and filtered data
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 514075 entries, 0 to 1368549
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         514075 non-null  object 
 1   averageRating  514075 non-null  float64
 2   numVotes       514075 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.7+ MB


## Save the data

In [64]:
## Create a Data folder
os.makedirs('Data/', exist_ok=True) 

In [65]:
## Save current dataframes to files
akas.to_csv("Data/title_akas.csv.gz", compression='gzip', index=False)
basics.to_csv("Data/title_basics.csv.gz", compression='gzip', index=False)
ratings.to_csv("Data/title_ratings.csv.gz", compression='gzip', index=False)

## Preview saved data

In [66]:
akas_clean = pd.read_csv("Data/title_akas.csv.gz")
akas_clean.info()
print(f'There are {akas_clean.shape[0]} rows and {akas_clean.shape[1]} columns in the clean and filtered title_akas dataset.')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1477044 entries, 0 to 1477043
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1477044 non-null  object
 1   ordering         1477044 non-null  int64 
 2   title            1477044 non-null  object
 3   region           1477044 non-null  object
 4   language         4208 non-null     object
 5   types            985576 non-null   object
 6   attributes       47725 non-null    object
 7   isOriginalTitle  1477044 non-null  bool  
dtypes: bool(1), int64(1), object(6)
memory usage: 80.3+ MB
There are 1477044 rows and 8 columns in the clean and filtered title_akas dataset.


In [68]:
basics_clean = pd.read_csv("Data/title_basics.csv.gz")
basics_clean.info()
print(f'There are {basics_clean.shape[0]} rows and {basics_clean.shape[1]} columns in the clean and filtered title_basics dataset.')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82235 entries, 0 to 82234
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          82235 non-null  object 
 1   titleType       82235 non-null  object 
 2   primaryTitle    82235 non-null  object 
 3   originalTitle   82235 non-null  object 
 4   isAdult         82235 non-null  bool   
 5   startYear       82235 non-null  int64  
 6   endYear         0 non-null      float64
 7   runtimeMinutes  82235 non-null  int64  
 8   genres          82235 non-null  object 
dtypes: bool(1), float64(1), int64(2), object(5)
memory usage: 5.1+ MB
There are 82235 rows and 9 columns in the clean and filtered title_basics dataset.


In [69]:
ratings_clean = pd.read_csv("Data/title_ratings.csv.gz")
ratings_clean.info()
print(f'There are {ratings_clean.shape[0]} rows and {ratings_clean.shape[1]} columns in the clean and filtered title_ratings dataset.')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514075 entries, 0 to 514074
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         514075 non-null  object 
 1   averageRating  514075 non-null  float64
 2   numVotes       514075 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 11.8+ MB
There are 514075 rows and 3 columns in the clean and filtered title_ratings dataset.
